# Viewer
For looking at the results of `space-between-expt.ipynb` .

If you didn't run this notebook you can grab a copy by running `$ make pulls3-results` to grab a copy from S3 - this will put results into `energypy

## Load up all results

In [ ]:
!pip install matplotlib pandas -Uq
import pandas as pd

results = pd.read_parquet('./results/monthly-results.parquet')
results.head(3)

## Entire time period summary

In [ ]:
summary = results.groupby('objective').agg({
    'Actual Cost [$/5T]': 'sum',
    'Carbon Cost [tC/5T]': 'sum',
})
summary.loc['delta', :] = summary.loc['price', :] - summary.loc['carbon', :]
summary['carbon-price'] = summary.iloc[:, 0] / summary.iloc[:, 1]
summary.head()

## Monthly Summaries

In [ ]:
pvt = results.pivot_table(index='date', values=['Actual Cost [$/5T]', 'Carbon Cost [tC/5T]'], columns='objective')
pvt.columns = [' '.join(col).strip() for col in pvt.columns.values]
pvt.head(3)

In [ ]:
pvt['price-delta'] = pvt.loc[:, 'Actual Cost [$/5T] carbon'] - pvt.loc[:, 'Actual Cost [$/5T] price']
pvt['carbon-delta'] = pvt.loc[:, 'Carbon Cost [tC/5T] price'] - pvt.loc[:, 'Carbon Cost [tC/5T] carbon']
pvt['carbon-price'] = pvt['price-delta'] / pvt['carbon-delta']
pvt

In [ ]:
import matplotlib.pyplot as plt

f, axes = plt.subplots(nrows=3, figsize=(20, 10), sharex=True)
pvt.plot(ax=axes[0], y='price-delta', kind='bar')
pvt.plot(ax=axes[1], y='carbon-delta', kind='bar')
pvt.plot(ax=axes[2], y='carbon-price', kind='bar')

In [ ]:
#  annual groupby
pvt.index = pd.to_datetime(pvt.index)

annual = pvt.groupby(pvt.index.year).agg({
    'price-delta': 'sum',
    'carbon-delta': 'sum'
})
annual['carbon-price'] = annual['price-delta'] / annual['carbon-delta']

f, axes = plt.subplots(nrows=3, figsize=(20, 10), sharex=True)
annual.plot(ax=axes[0], y='price-delta', kind='bar')
annual.plot(ax=axes[1], y='carbon-delta', kind='bar')
annual.plot(ax=axes[2], y='carbon-price', kind='bar')

In [ ]:
#  scatter carbon delta versus price delta
pvt.plot(x='price-delta', y='carbon-delta', kind='scatter')